### Liens python :

- [docs.python](https://docs.python.org)
- [python.doctor](https://python.doctor)

### Projet :
- [CVE dataset](https://nvd.nist.gov/vuln/data-feeds)

- [CVE descriptif](https://www.redhat.com/fr/topics/security/what-is-cve)

- [CPE descriptif](https://medium.com/prohacktive/comment-exploiter-la-base-cve-du-nist-dfb10837da5c)

- [Standards pour la gestion des vulnérabilités](https://www.cert-ist.com/public/fr/SO_detail?code=standards_gestion_vulnerabilites)

### à voir :

- cartopy
- ipywidget
- [kaggle](https://www.kaggle.com/)

### notes :

***CWE*** est un système de catégorisation des types de vulnérabilité, tandis que ***CVE*** est une référence à une vulnérabilité spécifique. 

files.maxMemoryForLargeFilesMB


<hr>

<h2>Introduction</h2>

**Common Vulnerabilities and Exposures** ou **CVE** est un dictionnaire des informations publiques relatives aux vulnérabilités de sécurité. Le dictionnaire est maintenu par l'organisme MITRE, soutenu par le département de la Sécurité intérieure des États-Unis.

**Common Vulnerability Scoring System** ou **CVSS** est un système d'évaluation standardisé de la criticité des vulnérabilités selon des critères objectifs et mesurables.


CVSS se compose de trois groupes de mesures : Base, Temporel, et Environnemental. 
Les métriques de base produisent un score allant de 0 à 10, qui peut ensuite être modifié en notant les métriques temporelles et environnementales.
<br>
Un score CVSS est également représenté sous la forme d'une chaîne vectorielle, une représentation textuelle comprimée des valeurs utilisées pour obtenir le score. Ainsi, CVSS est bien adapté comme système de mesure standard pour les industries, les organisations et les gouvernements qui ont besoin de scores de gravité de vulnérabilité précis et cohérents.
<br>

La base de données nationale sur les vulnérabilités (**NVD**) fournit des scores CVSS pour presque toutes les vulnérabilités connues.
<br>

**La NVD prend en charge les normes CVSS v2.0 et v3.X.**

![cvss](media/cvss.png)

Le NVD fournit des "scores de base" CVSS qui représentent les caractéristiques innées de chaque vulnérabilité.
Le NVD ne fournit pas actuellement de "scores temporels" (mesures qui évoluent dans le temps en raison d'événements extérieurs à la vulnérabilité) ou de "scores environnementaux" (scores personnalisés pour refléter l'impact de la vulnérabilité sur votre organisation). Cependant, le NVD fournit un calculateur CVSS pour CVSS v2 et v3 qui vous permet d'ajouter des données de score temporel et environnemental.
<br>

Pour certaines vulnérabilités, toutes les informations nécessaires à la création des scores CVSS peuvent ne pas être disponibles. Cela se produit généralement lorsqu'un fournisseur annonce une vulnérabilité mais refuse de fournir certains détails. Dans de telles situations, les analystes du NVD attribuent des scores CVSS en utilisant l'approche du pire cas. Ainsi, si un fournisseur ne fournit aucun détail sur une vulnérabilité, le NVD attribuera à cette vulnérabilité une note de 10.0 (la note la plus élevée).



In [ ]:
import json
from glob import glob
import gzip
import pandas as pd
from pandas import json_normalize
import requests
from datetime import datetime
import xml.etree.ElementTree as et


In [ ]:
# Download all cve from nvd.nist.gov
url = 'https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-{}.json.gz'

for cve in range(2002, datetime.now().year+1):
    data = requests.get(url.format(cve)).content
    with open(f'cve_dataset/nvdcve-1.1-{cve}.json.gz','wb') as f:
        f.write(data)

In [1]:
# Use Common Weakness Enumeration for type categorisation
cwe_dataset = dict()
tree = et.parse('cve_dataset/cwec_v4.7.xml')
root = tree.getroot()

for weakness in root.iter('{http://cwe.mitre.org/cwe-6}Weakness'):
    cwe_dataset[weakness.get('ID')] = weakness.get('Name')

CWE_df = pd.DataFrame.from_dict(cwe_dataset, orient='index', columns=["CWE_NAME"])

In [ ]:
CWE_df

In [2]:
# Extract gzip data and create dataframe
cve_dataset = list()

for data in glob('cve_dataset/*.gz'):
    with open(data,'rb') as f:
        json_data = json.loads(gzip.decompress(f.read()))

        cve_items = json_normalize(json_data['CVE_Items'])
        cve_items.drop([
            'cve.data_type',
            'cve.data_format',
            'cve.data_version',
            # 'cve.problemtype.problemtype_data',
            'cve.references.reference_data',
            'configurations.CVE_data_version',
            'configurations.nodes',
            'cve.description.description_data'],
            axis=1,
            inplace=True
        )

        descriptions = json_normalize(
            json_data['CVE_Items'],
            record_path=[['cve','description','description_data']],
            meta=[['cve','CVE_data_meta','ID']]
        )

        descriptions.drop(['lang'],axis=1,inplace=True)
        descriptions.rename(columns={"value":"description"}, inplace=True)

        dataframe = cve_items.merge(descriptions,on='cve.CVE_data_meta.ID')

        dataframe.rename(columns={"cve.CVE_data_meta.ID":"ID"}, inplace=True)
        dataframe = dataframe.set_index("ID")
        
        cve_dataset.append(dataframe)

df = pd.concat(cve_dataset)
df = df[sorted(df)]


In [3]:
# df.sample(10)

# for d in range(200,250):

#     print(df['cve.problemtype.problemtype_data'][d][0]['description'][0]['value'])

cwe_id =df['cve.problemtype.problemtype_data'].ravel()


In [ ]:


df.info()

In [ ]:
pd.options.display.max_columns = None

# # cast date
df['publishedDate'] = pd.to_datetime(df['publishedDate'])
df['lastModifiedDate'] = pd.to_datetime(df['lastModifiedDate'])


In [ ]:
# displays statistics for quantitative variables
df.describe()

In [ ]:
# find by CVE-id
df.loc['CVE-2005-1479']

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df.duplicated().sum()

# ** REJECT ** DO NOT USE THIS CANDIDATE NUMBER
mask = df.duplicated()

df[mask]

In [ ]:
# test graph
years = df['publishedDate'].dt.year

years[years == 1989]
# a = years.value_counts().sort_index()

# years.value_counts().sort_index().plot(kind='bar',figsize=(15, 5),logy=True)
